# Entrenando y evaluando modelos

Este notebook es una adaptación del [original de *Aurélien Gerón*](https://github.com/ageron/handson-ml3/blob/main/02_end_to_end_machine_learning_project.ipynb), de su libro: [Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow, 3rd Edition. Aurélien Géron](https://www.oreilly.com/library/view/hands-on-machine-learning/9781098125967/)

## Importación y muestreo de datos

In [36]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

housing = pd.read_csv("./data/housing.csv")

# X = housing.drop(columns="median_house_value")
# y = housing["median_house_value"]

## Preprocesado

Se incorpora el preprocesado completo que se realiza en el libro original, aunque varios de los pasos no los hemos abordado por su complejidad.
<!-- TODO: el resultado no es el mismo, faltan las columnas combinadas al menos -->

In [37]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.cluster import KMeans

In [38]:
cat_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OneHotEncoder(handle_unknown="ignore"))

In [39]:
class ClusterSimilarity(BaseEstimator, TransformerMixin):
    def __init__(self, n_clusters=10, gamma=1.0, random_state=None):
        self.n_clusters = n_clusters
        self.gamma = gamma
        self.random_state = random_state

    def fit(self, X, y=None, sample_weight=None):
        self.kmeans_ = KMeans(self.n_clusters, n_init=10,
                              random_state=self.random_state)
        self.kmeans_.fit(X, sample_weight=sample_weight)
        return self  # always return self!

    def transform(self, X):
        return rbf_kernel(X, self.kmeans_.cluster_centers_, gamma=self.gamma)
    
    def get_feature_names_out(self, names=None):
        return [f"Cluster {i} similarity" for i in range(self.n_clusters)]

cluster_simil = ClusterSimilarity(n_clusters=10, gamma=1., random_state=42)

In [40]:
def column_ratio(X):
    return X[:, [0]] / X[:, [1]]

def ratio_name(function_transformer, feature_names_in):
    return ["ratio"]  # feature names out

def ratio_pipeline():
    return make_pipeline(
        SimpleImputer(strategy="median"),
        FunctionTransformer(column_ratio, feature_names_out=ratio_name),
        StandardScaler())

log_pipeline = make_pipeline(
    SimpleImputer(strategy="median"),
    FunctionTransformer(np.log, feature_names_out="one-to-one"),
    StandardScaler())

default_num_pipeline = make_pipeline(SimpleImputer(strategy="median"),
                                     StandardScaler())
preprocessing = ColumnTransformer([
        ("bedrooms", ratio_pipeline(), ["total_bedrooms", "total_rooms"]), # razón entre total_bedrooms y total_rooms
        ("rooms_per_house", ratio_pipeline(), ["total_rooms", "households"]), # razón entre total_rooms y households
        ("people_per_house", ratio_pipeline(), ["population", "households"]), # razón entre population y households
        ("log", log_pipeline, ["total_bedrooms", "total_rooms", "population",
                               "households", "median_income"]), # logaritmo de las columnas seleccionadas
        ("geo", cluster_simil, ["latitude", "longitude"]), # similitud con los clusters
        ("cat", cat_pipeline, make_column_selector(dtype_include=object)), # pipeline categórico
    ],
    remainder=default_num_pipeline)  # one column remaining: housing_median_age

## Entrenando y evaluando modelos

In [41]:
X = housing.drop(columns="median_house_value")
y = housing["median_house_value"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:
from sklearn.linear_model import LinearRegression

lin_reg = make_pipeline(preprocessing, LinearRegression()) # creamos un pipeline con el preprocesado y el modelo
lin_reg.fit(X_train, y_train) # preprocesamos los datos de entrenamiento y entrenamos el modelo

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder=Pipeline(steps=[('simpleimputer',
                                                              SimpleImputer(strategy='median')),
                                                             ('standardscaler',
                                                              StandardScaler())]),
                                   transformers=[('bedrooms',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('functiontransformer',
                                                                   FunctionTransformer(feature_names_out=<function ratio_name at 0x7f2...
                                                   'median_income']),
                                                 ('geo',
                                                  ClusterSimilarity(random_state=42),
                                                  ['latitude', 'longitude']),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f2d35c35040>)])),
                ('linearregression', LinearRegression())])

In [43]:
y_pred = lin_reg.predict(X_test) # Realizamos predicciones con los datos de test

Ahora podemos comparar algunos de los resultados predecidos con sus etiquetas reales:

In [44]:
print("Valores reales:", list(y_test.iloc[:10]))
print("Predicciones:", list(y_pred[:10].round(-2)))

Valores reales: [47700.0, 45800.0, 500001.0, 218600.0, 278000.0, 158700.0, 198200.0, 157500.0, 340000.0, 446600.0]
Predicciones: [13400.0, 170900.0, 351700.0, 306700.0, 275500.0, 254400.0, 292000.0, 232400.0, 304600.0, 366400.0]


y ver el error porcentual en estas predicciones:

In [45]:
error_ratios = y_pred.round(-2) / y_test - 1
print(", ".join([f"{100 * ratio:.1f}%" for ratio in error_ratios]))

-71.9%, 273.1%, -29.7%, 40.3%, -0.9%, 60.3%, 47.3%, 47.6%, -10.4%, -18.0%, 355.0%, -18.6%, -10.2%, 11.8%, 41.9%, 123.3%, 152.2%, 243.4%, 60.3%, -30.7%, -9.8%, -21.5%, 33.9%, -15.4%, 25.3%, -51.6%, 51.4%, 449.8%, 8.0%, 0.7%, 155.2%, 7.3%, 52.0%, -26.0%, -7.7%, 114.9%, 49.7%, 18.5%, 37.9%, 114.9%, 59.8%, 70.1%, 799.6%, 23.1%, 24.9%, -13.1%, 42.5%, -18.9%, 4.4%, 146.9%, -16.5%, 31.2%, 43.4%, -79.6%, 7.5%, 40.1%, -17.8%, 32.8%, 7.7%, 72.5%, -37.4%, 30.7%, -17.8%, 57.5%, 25.0%, 30.0%, -28.1%, 97.6%, -49.0%, 364.2%, 168.7%, 67.7%, 41.0%, 61.8%, 4.7%, 46.3%, 7.4%, 14.9%, 91.7%, 80.4%, 46.2%, 52.9%, -18.9%, -7.6%, -7.7%, 8.5%, 12.9%, 48.0%, -2.3%, -6.3%, 121.1%, 12.0%, 468.8%, -132.7%, -12.0%, -11.4%, -2.4%, -36.6%, 252.2%, -0.6%, 68.8%, -0.8%, -43.2%, 57.1%, 39.7%, -7.2%, 63.1%, 52.5%, 21.3%, 31.0%, 58.1%, 37.4%, 1334.7%, -28.5%, 86.5%, 100.4%, 142.7%, 36.2%, 3.7%, 12.0%, 64.3%, 73.0%, 2.3%, -26.2%, 25.2%, 94.2%, -48.2%, 15.5%, 81.2%, 161.8%, 30.1%, -11.5%, 8.2%, 501.0%, 35.8%, 54.8%, 93.6%, 

pero podemos ver el rendimiento con el error cuadrático medio, como habíamos estipulado:

In [46]:
from sklearn.metrics import root_mean_squared_error
root_mean_squared_error(y_test, y_pred)

87533.13725738808

Un error de 87533$ para predicciones del valor de viviendas cuyo precio medio es de 206856$ no parece muy útil.

Podemos probar otro modelo:

In [47]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = make_pipeline(preprocessing, DecisionTreeRegressor(random_state=42))
tree_reg.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder=Pipeline(steps=[('simpleimputer',
                                                              SimpleImputer(strategy='median')),
                                                             ('standardscaler',
                                                              StandardScaler())]),
                                   transformers=[('bedrooms',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('functiontransformer',
                                                                   FunctionTransformer(feature_names_out=<function ratio_name at 0x7f2...
                                                  ClusterSimilarity(random_state=42),
                                                  ['latitude', 'longitude']),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f2d35c35040>)])),
                ('decisiontreeregressor',
                 DecisionTreeRegressor(random_state=42))])

In [48]:
y_pred = tree_reg.predict(X_test)
root_mean_squared_error(y_test, y_pred)

68274.71567413137

Vemos que el error sigue siendo demasiado alto para considerar que tenemos un modelo útil (una estimación de precio de vivienda con un error de 68000$ es probablemente mucho peor que la que haría cualquier persona con un conocimiento básico del mercado inmobiliario).

## *Cross-validation*

Hasta ahora hemos separado en dos conjuntos el de entrenamiento y el de test. Sin embargo, en muchos casos, el rendimiento variará dependiendo del muestreo que hayamos hecho. Si dejamos de fijar la semilla de muestreo (parámetro `random_state` de la función `train_test_split`), obtendremos resultados distintos (si bien en este caso no varían demasiado en ninguno de los dos modelos).

Una forma más eficiente de hacerlo es la **validación cruzada o *cross-validation***: en lugar de dividir el conjunto de entrenamiento en dos, se divide en *k* conjuntos (*folds*). Luego se entrena el modelo *k* veces, cada vez dejando un conjunto distinto como conjunto de validación y los otros *k-1* como conjunto de entrenamiento. El resultado es un array con *k* puntuaciones. 

Por ejemplo, con el siguiente código realiza el entrenamiento con 10 muestreos distintos. Los resultados serán similares a los que podríamos optener ejecutando 10 veces el código sin fijar la semilla de muestreo aleatorio. La contrapartida es evidente: el coste computacional también se multiplica por 10.

Aparece así el concepto de **conjunto de validación**. El conjunto de validación se utiliza para comparar modelos y los ajustes de hiperparámetros, y es el que va cambiándo en cada itearación e la validación cruzada. El conjunto de test se guarda para evaluar el rendimiento final del modelo elegido una vez que se ha entrenado.

[<img src="./data/cross-validation.png" width="500">](https://www.researchgate.net/figure/Train-test-cross-validation-split-methodology-used-in-this-paper-The-first-operation_fig2_340567535)

[<img src="./data/cross-validation2.png" width="700">](https://www.statology.org/validation-set-vs-test-set/)

In [49]:
from sklearn.model_selection import cross_val_score

tree_rmses = -cross_val_score(estimator = tree_reg, 
                              X = X_train,
                              y = y_train,
                              scoring = "neg_root_mean_squared_error",
                              cv = 10)

print(tree_rmses)
pd.Series(tree_rmses).describe()

[63389.44938725 68140.47374293 67334.7158258  70064.51809136
 67809.35419491 65465.57865981 61882.4062051  68061.13675185
 68303.14187137 63707.44822114]


count       10.000000
mean     66415.822295
std       2651.871089
min      61882.406205
25%      64146.980831
50%      67572.035010
75%      68120.639495
max      70064.518091
dtype: float64

El parámetro `scoring` de la función `cross_val_score` espera una **función de utilidad** (mayor es mejor) en lugar de una **función de coste** (menor es mejor), por lo que la puntuación es en realidad el opuesto del RMSE. Es un valor negativo, por lo que necesitamos cambiar el signo de la salida para obtener los valores de RMSE.

Podemos ver que recuperamos un RMSE medio de 66415$ con una desviación estándar de 2651$. Nos aporta información más detallada (y menos dependiente del muestreo que hayamos hecho) sobre el rendimiento.

## Probando con otro modelo (*Random Forest*)

In [50]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = make_pipeline(preprocessing, RandomForestRegressor(random_state=42))
forest_rmses = -cross_val_score(forest_reg, X_train, y_train, scoring = "neg_root_mean_squared_error", cv = 10)
pd.Series(forest_rmses).describe()

count       10.000000
mean     46976.553291
std       2058.981796
min      44106.745679
25%      45121.642231
50%      47028.117967
75%      48920.501984
max      49570.985996
dtype: float64

Random Forest consigue una mejoría (46976$ de error medio).

Aunque sigue siendo un error alto es el mejor modelo que tenemos hasta ahora. Asumiendo que nos quedemos con él, podemos finalmente entrenar el modelo elegido con el conjunto de entrenamiento entero y ver su rendimiento con el conjunto de test que mantuvimos separado.
<!-- En qué casos el resultado de test va a ser distinto al de cross validation? Si este resultado no se compara, por qué es una ventaja que sea independiente del sesgo de validación, si por contra recupera el de muestreo -->

In [51]:
y_pred = forest_reg.fit(X_train, y_train).predict(X_test)
forest_rmse = root_mean_squared_error(y_test, y_pred)
forest_rmse

47672.04377793412